<h2>Segmenting and Clustering Neighborhoods in Toronto</h2>


Lets fetch all necessary libraries

In [3]:
import pandas as pd
import numpy as np
import bs4
import requests
import lxml
import html5lib
from pandas.io.json import json_normalize

<h2>Get data from wiki webpage</h2>

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
soup = bs4.BeautifulSoup(source, 'html.parser')

<h2>Scraping html from wiki page using beautiful soup</h2>

In [6]:
PostalCode = []
Borough = []
Neighborhood = []

for data in soup.tbody.find_all('tr')[1:]:
    PostalCode.append(data.find('td').text)
    Borough.append(data.find_all('td')[1].text)
    Neighborhood.append(data.find_all('td')[2].text[:-1])

In [7]:
data = {'PostalCode': PostalCode,
        'Borough': Borough,
        'Neighborhood': Neighborhood}
df = pd.DataFrame(data)
# Drop rows where Borough is 'Not assigned', then reset index
df = df[df.Borough != 'Not assigned'].reset_index().drop('index', axis = 1)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<h2>Combining neighborhouds with the same postal codes <h2/>

In [8]:
i = 1
while(i < len(df)):
    if df['PostalCode'].iloc[i] == df['PostalCode'].iloc[i - 1]:
        df.at[i - 1, 'Neighborhood'] = df.Neighborhood.iloc[i - 1] +', ' + df.Neighborhood.iloc[i]
        df.drop(index = i, inplace = True)
        df = df.reset_index().drop('index', axis = 1)
    else:
        i += 1

In [9]:
df.shape

(103, 3)

<h3>Above shape is submission for part 1 of peer review</h3>

<h2>Part2</h2>
<h2>Merging initial table with Geospacial Coordinates</h2>

In [10]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Reading csv file

In [11]:
Geo = pd.read_csv("Geospatial_Coordinates.csv") 
Geo.rename({'Postal Code': 'PostalCode'}, axis='columns', inplace=True)

In [12]:
# Merge Latitude and longitude to original dataframe
df = pd.merge(df, Geo, on='PostalCode')


In [18]:
df.head(11)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


<h2>Third part -lets create some maps</h2>

In [19]:
import folium
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[43.706204, -79.398752], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#e182f2',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
# Show map Toronto with borough and neighborhood 
map_Toronto

**Defining Fousquare Credentials and date for version**

In [20]:

CLIENT_ID = 'GHHU0AV2RH5UJL1LTLSCN0G2ZCONUY0OSVPMYDFDIEBJBYYG' # your Foursquare ID
CLIENT_SECRET = '1R44PQXYZDOZDGJS3B33VXUDWUFP5HDDN14QWO5R3SC4CBA4' # your Foursquare Secret
VERSION = '20190217' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GHHU0AV2RH5UJL1LTLSCN0G2ZCONUY0OSVPMYDFDIEBJBYYG
CLIENT_SECRET:1R44PQXYZDOZDGJS3B33VXUDWUFP5HDDN14QWO5R3SC4CBA4


In [21]:
neighborhood_latitude = df.loc[95, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[95, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[95, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Upper Rouge are 43.836124700000006, -79.20563609999999.


In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000# define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=GHHU0AV2RH5UJL1LTLSCN0G2ZCONUY0OSVPMYDFDIEBJBYYG&client_secret=1R44PQXYZDOZDGJS3B33VXUDWUFP5HDDN14QWO5R3SC4CBA4&v=20190217&ll=43.836124700000006,-79.20563609999999&radius=2000&limit=100'

In [23]:
results = requests.get(url).json()

In [24]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Pumpkinland (Whittamore Farms),Farm,43.849236,-79.206647
1,Rouge Park - Finch Meander Trail,Trail,43.824850,-79.193596
2,Harvest Moon Park,Playground,43.821684,-79.211021
3,Clownfish Village,Sculpture Garden,43.821000,-79.205396
4,Taj supermarket,Grocery Store,43.818746,-79.210172
5,Cedar Brae Golf and Country Club,Golf Course,43.839154,-79.230089


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


<h2>Exploring neighborhoods in Toronto</h2>

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
       
    return(nearby_venues)

In [30]:
Toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                 latitudes=df['Latitude'],
                                 longitudes=df['Longitude'],
                                 radius = 2000
                                )

In [31]:
print(Toronto_venues.shape)
Toronto_venues.head()

(8497, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,Darband Restaurant,43.755194,-79.348498,Middle Eastern Restaurant
3,Parkwoods,43.753259,-79.329656,LA Fitness,43.747665,-79.347077,Gym / Fitness Center
4,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park


In [32]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 327 uniques categories.


<h2>Analysing neighbourhoods</h2>
<h3>creating one hot file with dummy values by venue category</h3>

In [35]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Vineyard,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,Vineyard,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0


In [37]:
Toronto_grouped.shape

(103, 327)

In [38]:
Toronto_onehot.shape

(8497, 327)

In [42]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.05
1            Hotel  0.05
2          Theater  0.04
3             Café  0.04
4  Thai Restaurant  0.03


----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.13
1           Coffee Shop  0.07
2              Pharmacy  0.05
3        Sandwich Place  0.03
4  Fast Food Restaurant  0.03


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue  freq
0    Chinese Restaurant  0.21
1           Coffee Shop  0.07
2           Pizza Place  0.05
3                  Park  0.03
4  Hong Kong Restaurant  0.03


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Coffee Shop  0.20
1  Fast Food Restaurant  0.12
2           Pizza Place  0.10
3     Indian Restaurant  0.07
4         Grocery Store  0.07


----Alderwood, Long Branch----
                  venue  freq
0      

                   venue  freq
0  Vietnamese Restaurant  0.15
1            Pizza Place  0.12
2                   Park  0.08
3          Grocery Store  0.08
4         Cosmetics Shop  0.04


----East Birchmount Park, Ionview, Kennedy Park----
                  venue  freq
0           Coffee Shop  0.07
1         Grocery Store  0.07
2    Chinese Restaurant  0.05
3  Fast Food Restaurant  0.05
4              Pharmacy  0.05


----East Toronto----
              venue  freq
0  Greek Restaurant  0.11
1              Café  0.06
2            Bakery  0.05
3       Pizza Place  0.04
4         Gastropub  0.03


----Emery, Humberlea----
            venue  freq
0     Coffee Shop  0.10
1   Grocery Store  0.10
2  Sandwich Place  0.05
3            Bank  0.05
4      Beer Store  0.05


----Fairview, Henry Farm, Oriole----
                       venue  freq
0                Coffee Shop  0.13
1             Clothing Store  0.06
2                   Pharmacy  0.05
3             Sandwich Place  0.05
4  Middle Easter

                 venue  freq
0          Coffee Shop  0.12
1   Italian Restaurant  0.08
2                 Café  0.04
3  Japanese Restaurant  0.03
4               Bakery  0.03


----Rouge, Malvern----
                  venue  freq
0           Zoo Exhibit  0.35
1  Fast Food Restaurant  0.09
2           Pizza Place  0.05
3                   Zoo  0.05
4           Bus Station  0.02


----Runnymede, Swansea----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.06
2              Bakery  0.06
3                Park  0.06
4  Italian Restaurant  0.05


----Ryerson, Garden District----
                venue  freq
0                Café  0.05
1         Coffee Shop  0.05
2           Gastropub  0.05
3     Thai Restaurant  0.04
4  Italian Restaurant  0.03


----Scarborough Village----
                  venue  freq
0  Fast Food Restaurant  0.13
1           Coffee Shop  0.10
2           Pizza Place  0.07
3              Pharmacy  0.06
4        Sandwich Place  0.06


----Silve

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [39]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Hotel,Café,Theater,Concert Hall,Restaurant,Thai Restaurant,Pizza Place,Sporting Goods Shop,Movie Theater
1,Agincourt,Chinese Restaurant,Coffee Shop,Pharmacy,Sandwich Place,Cantonese Restaurant,Japanese Restaurant,Restaurant,Indian Restaurant,Fast Food Restaurant,Pizza Place
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Coffee Shop,Pizza Place,Bakery,Japanese Restaurant,Park,Hong Kong Restaurant,Dessert Shop,Noodle House,Vietnamese Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Fast Food Restaurant,Pizza Place,Indian Restaurant,Grocery Store,Sandwich Place,Steakhouse,Skating Rink,Bank,Thrift / Vintage Store
4,"Alderwood, Long Branch",Coffee Shop,Burger Joint,Fast Food Restaurant,Pizza Place,Pharmacy,Breakfast Spot,Department Store,Electronics Store,Seafood Restaurant,Sandwich Place


<h2>Cluster neighbourhoods</h2>

In [41]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [44]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:]

array([1, 3, 3, 4, 4, 3, 3, 2, 1, 3, 3, 2, 2, 4, 2, 2, 2, 4, 4, 1, 2, 2,
       1, 4, 3, 3, 3, 1, 2, 2, 2, 4, 1, 2, 4, 2, 3, 4, 3, 4, 2, 4, 4, 1,
       4, 2, 4, 3, 2, 1, 2, 2, 4, 3, 2, 2, 3, 2, 3, 3, 4, 3, 4, 2, 2, 2,
       4, 4, 2, 1, 2, 4, 4, 1, 2, 4, 2, 2, 2, 2, 1, 4, 4, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 4, 0, 4, 4, 4, 1, 3, 4, 3, 2, 4])

In [45]:
Toronto_merged = df

# add clustering labels
Toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Coffee Shop,Pharmacy,Japanese Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Supermarket,Park,Bank,Shopping Mall
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Coffee Shop,Fast Food Restaurant,Gym,Clothing Store,Grocery Store,Sandwich Place,Discount Store,Middle Eastern Restaurant,Japanese Restaurant,Beer Store
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,3,Coffee Shop,Café,Italian Restaurant,Park,Restaurant,Pub,Gastropub,Bakery,Japanese Restaurant,Thai Restaurant
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,4,Clothing Store,Coffee Shop,Fast Food Restaurant,Vietnamese Restaurant,Grocery Store,Jewelry Store,Fried Chicken Joint,Discount Store,Sporting Goods Shop,Dessert Shop
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494,4,Café,Japanese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Park,Gastropub,Bookstore,Concert Hall,Pizza Place


In [47]:
# create map
map_clusters = folium.Map(location=[43.706204, -79.398752], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [48]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,Etobicoke,0,Coffee Shop,Burger Joint,Fast Food Restaurant,Pizza Place,Pharmacy,Breakfast Spot,Department Store,Electronics Store,Seafood Restaurant,Sandwich Place


In [49]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Coffee Shop,Pharmacy,Japanese Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Supermarket,Park,Bank,Shopping Mall
8,East York,1,Pizza Place,Fast Food Restaurant,Park,Sandwich Place,Coffee Shop,Grocery Store,Bakery,Beer Store,Furniture / Home Store,Bank
19,East Toronto,1,Coffee Shop,Pub,Beach,Breakfast Spot,Café,Bakery,Park,Bar,Thai Restaurant,Ice Cream Shop
22,Scarborough,1,Coffee Shop,Fast Food Restaurant,Pizza Place,Pharmacy,Beer Store,Supermarket,Chinese Restaurant,Furniture / Home Store,Sandwich Place,Indian Restaurant
27,North York,1,Coffee Shop,Chinese Restaurant,Bank,Sandwich Place,Park,Pharmacy,Japanese Restaurant,Pizza Place,Bakery,Asian Restaurant
32,Scarborough,1,Fast Food Restaurant,Coffee Shop,Pizza Place,Pharmacy,Sandwich Place,Grocery Store,Beer Store,Big Box Store,Liquor Store,Theater
43,West Toronto,1,Café,Coffee Shop,Bar,Restaurant,Vegetarian / Vegan Restaurant,Hotel,New American Restaurant,Tea Room,Bakery,Ice Cream Shop
49,North York,1,Coffee Shop,Vietnamese Restaurant,Fast Food Restaurant,Pizza Place,Grocery Store,Supermarket,Bank,Sandwich Place,Furniture / Home Store,Pharmacy
69,West Toronto,1,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Brewery,Gastropub,Pizza Place,Park,Dessert Shop
73,Central Toronto,1,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Bakery,Pizza Place,Burger Joint,Japanese Restaurant,Yoga Studio,Bookstore


In [50]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,North York,2,Coffee Shop,Japanese Restaurant,Park,Restaurant,Supermarket,Pizza Place,Bank,Burger Joint,Italian Restaurant,Bar
11,Etobicoke,2,Coffee Shop,Convenience Store,Sandwich Place,Grocery Store,Park,Bank,Pizza Place,Pharmacy,Sushi Restaurant,Gym
12,Scarborough,2,Coffee Shop,Breakfast Spot,Sandwich Place,Gym,Liquor Store,Supermarket,Discount Store,Fast Food Restaurant,Bank,Bakery
14,East York,2,Park,Café,Coffee Shop,Pizza Place,Gastropub,Thai Restaurant,Bakery,Ethiopian Restaurant,Ice Cream Shop,Bar
15,Downtown Toronto,2,Coffee Shop,Café,Hotel,Thai Restaurant,Gastropub,Italian Restaurant,Farmers Market,Concert Hall,Theater,Cocktail Bar
16,York,2,Coffee Shop,Italian Restaurant,Bakery,Bank,Café,Mexican Restaurant,Pizza Place,Thai Restaurant,Caribbean Restaurant,Indian Restaurant
20,Downtown Toronto,2,Coffee Shop,Café,Hotel,Italian Restaurant,Park,Gastropub,Farmers Market,Restaurant,Steakhouse,Japanese Restaurant
21,York,2,Italian Restaurant,Coffee Shop,Bakery,Sandwich Place,Furniture / Home Store,Bank,Breakfast Spot,Mexican Restaurant,Grocery Store,Pizza Place
28,North York,2,Coffee Shop,Pizza Place,Pharmacy,Sandwich Place,Park,Deli / Bodega,Restaurant,Convenience Store,Bank,Fast Food Restaurant
29,East York,2,Coffee Shop,Sandwich Place,Restaurant,Park,Grocery Store,Bakery,Greek Restaurant,Japanese Restaurant,Burger Joint,Pharmacy


In [51]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,3,Coffee Shop,Fast Food Restaurant,Gym,Clothing Store,Grocery Store,Sandwich Place,Discount Store,Middle Eastern Restaurant,Japanese Restaurant,Beer Store
2,Downtown Toronto,3,Coffee Shop,Café,Italian Restaurant,Park,Restaurant,Pub,Gastropub,Bakery,Japanese Restaurant,Thai Restaurant
5,Etobicoke,3,Park,Grocery Store,Coffee Shop,Bank,Pizza Place,Pharmacy,Golf Course,Shopping Mall,Café,Liquor Store
6,Scarborough,3,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Zoo,Coffee Shop,Curling Ice,Liquor Store,Athletics & Sports,Grocery Store,Gym
9,Downtown Toronto,3,Café,Gastropub,Coffee Shop,Thai Restaurant,Italian Restaurant,Japanese Restaurant,Hotel,Ramen Restaurant,Pizza Place,Steakhouse
10,North York,3,Clothing Store,Coffee Shop,Furniture / Home Store,Fast Food Restaurant,Grocery Store,Fried Chicken Joint,Bakery,Cosmetics Shop,Toy / Game Store,Japanese Restaurant
24,Downtown Toronto,3,Coffee Shop,Café,Theater,Concert Hall,Pizza Place,Bookstore,Sandwich Place,Thai Restaurant,Tea Room,Sushi Restaurant
25,Downtown Toronto,3,Café,Coffee Shop,Korean Restaurant,Bar,Grocery Store,Vegetarian / Vegan Restaurant,Cocktail Bar,Ice Cream Shop,Restaurant,Indian Restaurant
26,Scarborough,3,Coffee Shop,Clothing Store,Gym,Restaurant,Wings Joint,Indian Restaurant,Pharmacy,Fast Food Restaurant,Bank,Sandwich Place
36,Downtown Toronto,3,Café,Hotel,Coffee Shop,Gym,Park,Japanese Restaurant,Steakhouse,Gym / Fitness Center,Farmers Market,Seafood Restaurant


In [52]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,4,Clothing Store,Coffee Shop,Fast Food Restaurant,Vietnamese Restaurant,Grocery Store,Jewelry Store,Fried Chicken Joint,Discount Store,Sporting Goods Shop,Dessert Shop
4,Queen's Park,4,Café,Japanese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Park,Gastropub,Bookstore,Concert Hall,Pizza Place
13,North York,4,Coffee Shop,Park,Japanese Restaurant,Asian Restaurant,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,American Restaurant,Turkish Restaurant
17,Etobicoke,4,Bank,Pharmacy,Grocery Store,Coffee Shop,Golf Course,Sandwich Place,Gym,Chinese Restaurant,Gym / Fitness Center,Flower Shop
18,Scarborough,4,Pizza Place,Coffee Shop,Park,Breakfast Spot,Fast Food Restaurant,Supermarket,Bank,Discount Store,Sandwich Place,Beer Store
23,East York,4,Coffee Shop,Indian Restaurant,Grocery Store,Supermarket,Burger Joint,Bakery,Restaurant,Sandwich Place,Dog Run,Turkish Restaurant
31,West Toronto,4,Café,Coffee Shop,Bar,Italian Restaurant,Park,Bakery,Brewery,Mexican Restaurant,Gastropub,Cocktail Bar
34,North York,4,Coffee Shop,Pizza Place,Sandwich Place,Fast Food Restaurant,Restaurant,Grocery Store,Bar,Japanese Restaurant,Italian Restaurant,Flea Market
37,West Toronto,4,Café,Bar,Italian Restaurant,Cocktail Bar,Pizza Place,Bakery,Yoga Studio,Sandwich Place,Coffee Shop,Asian Restaurant
39,North York,4,Chinese Restaurant,Pharmacy,Coffee Shop,Bank,Shopping Mall,Café,Park,Grocery Store,Sushi Restaurant,Japanese Restaurant
